# Lambda Functions and Pivot Tables

Until now, we have not made any changes or modifications to the data. In this section, we will:
* Use lambda functions to create new and alter existing columns
* Use pandas pivot tables as an alternative to ```df.groupby()``` to summarise data

Let's first read all the files and create a ```master_df```. 

In [1]:
# Loading libraries and files
import numpy as np
import pandas as pd

market_df = pd.read_csv("./global_sales_data/market_fact.csv")
customer_df = pd.read_csv("./global_sales_data/cust_dimen.csv")
product_df = pd.read_csv("./global_sales_data/prod_dimen.csv")
shipping_df = pd.read_csv("./global_sales_data/shipping_dimen.csv")
orders_df = pd.read_csv("./global_sales_data/orders_dimen.csv")

# Merging the dataframes to create a master_df
df_1 = pd.merge(market_df, customer_df, how='inner', on='Cust_id')
df_2 = pd.merge(df_1, product_df, how='inner', on='Prod_id')
df_3 = pd.merge(df_2, shipping_df, how='inner', on='Ship_id')
master_df = pd.merge(df_3, orders_df, how='inner', on='Ord_id')

master_df.head()

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin,...,Region,Customer_Segment,Product_Category,Product_Sub_Category,Order_ID_x,Ship_Mode,Ship_Date,Order_ID_y,Order_Date,Order_Priority
0,Ord_5446,Prod_16,SHP_7609,Cust_1818,136.81,0.01,23,-30.51,3.60,0.56,...,WEST,CORPORATE,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",36262,REGULAR AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED
1,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.69,0.00,26,1148.90,2.50,0.59,...,WEST,CORPORATE,TECHNOLOGY,TELEPHONES AND COMMUNICATION,36262,EXPRESS AIR,27-07-2010,36262,27-07-2010,NOT SPECIFIED
2,Ord_5446,Prod_6,SHP_7608,Cust_1818,164.02,0.03,23,-47.64,6.15,0.37,...,WEST,CORPORATE,OFFICE SUPPLIES,PAPER,36262,EXPRESS AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED
3,Ord_2978,Prod_16,SHP_4112,Cust_1088,305.05,0.04,27,23.12,3.37,0.57,...,ONTARIO,HOME OFFICE,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",37863,REGULAR AIR,26-02-2011,37863,24-02-2011,HIGH
4,Ord_5484,Prod_16,SHP_7663,Cust_1820,322.82,0.05,35,-17.58,3.98,0.56,...,WEST,CONSUMER,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",53026,REGULAR AIR,03-03-2012,53026,26-02-2012,LOW


### Lambda Functions

Say you want to create a new column indicating whether a given order was profitable or not (1/0). 

You need to apply a function which returns 1 if Profit > 0, else 0. This can be easily done using the ```apply()``` method on a column of the dataframe. 

In [2]:
# Create a function to be applied
def is_positive(x):
    return x > 0

# Create a new column
master_df['is_profitable'] = master_df['Profit'].apply(is_positive)
master_df.head()

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin,...,Customer_Segment,Product_Category,Product_Sub_Category,Order_ID_x,Ship_Mode,Ship_Date,Order_ID_y,Order_Date,Order_Priority,is_profitable
0,Ord_5446,Prod_16,SHP_7609,Cust_1818,136.81,0.01,23,-30.51,3.60,0.56,...,CORPORATE,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",36262,REGULAR AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED,False
1,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.69,0.00,26,1148.90,2.50,0.59,...,CORPORATE,TECHNOLOGY,TELEPHONES AND COMMUNICATION,36262,EXPRESS AIR,27-07-2010,36262,27-07-2010,NOT SPECIFIED,True
2,Ord_5446,Prod_6,SHP_7608,Cust_1818,164.02,0.03,23,-47.64,6.15,0.37,...,CORPORATE,OFFICE SUPPLIES,PAPER,36262,EXPRESS AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED,False
3,Ord_2978,Prod_16,SHP_4112,Cust_1088,305.05,0.04,27,23.12,3.37,0.57,...,HOME OFFICE,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",37863,REGULAR AIR,26-02-2011,37863,24-02-2011,HIGH,True
4,Ord_5484,Prod_16,SHP_7663,Cust_1820,322.82,0.05,35,-17.58,3.98,0.56,...,CONSUMER,OFFICE SUPPLIES,"SCISSORS, RULERS AND TRIMMERS",53026,REGULAR AIR,03-03-2012,53026,26-02-2012,LOW,False


In [7]:
master_df['Is_Profitable'] = master_df['Profit'].apply(lambda profit: profit > 0)
columns = ['Ord_id', 'Sales', 'Profit', 'Is_Profitable']
master_df[columns]

,Ord_id,Sales,Profit,Is_Profitable
0,Ord_5446,136.81,-30.51,False
1,Ord_5446,4701.69,1148.90,True
2,Ord_5446,164.02,-47.64,False
3,Ord_2978,305.05,23.12,True
4,Ord_5484,322.82,-17.58,False
...,...,...,...,...
8394,Ord_5018,7325.63,1899.23,True
8395,Ord_669,20872.16,-4437.91,False
8396,Ord_508,19109.61,-379.29,False
8397,Ord_3721,614.14,-735.27,False


In [8]:
master_df['Is_profit_btw_1000_and_2000'] = master_df['Profit'].apply(lambda profit: profit > 1000 and profit < 2000)
columns = ['Ord_id', 'Sales', 'Profit', 'Is_profit_btw_1000_and_2000']
master_df[columns]

,Ord_id,Sales,Profit,Is_profit_btw_1000_and_2000
0,Ord_5446,136.81,-30.51,False
1,Ord_5446,4701.69,1148.90,True
2,Ord_5446,164.02,-47.64,False
3,Ord_2978,305.05,23.12,False
4,Ord_5484,322.82,-17.58,False
...,...,...,...,...
8394,Ord_5018,7325.63,1899.23,True
8395,Ord_669,20872.16,-4437.91,False
8396,Ord_508,19109.61,-379.29,False
8397,Ord_3721,614.14,-735.27,False


In [9]:
master_df['has_loss'] = master_df['Profit'].apply(lambda data: data < 0)
columns = ['Ord_id','Sales', 'Profit', 'is_profitable', 'has_loss']
master_df.loc[:,columns]

,Ord_id,Sales,Profit,is_profitable,has_loss
0,Ord_5446,136.81,-30.51,False,True
1,Ord_5446,4701.69,1148.90,True,False
2,Ord_5446,164.02,-47.64,False,True
3,Ord_2978,305.05,23.12,True,False
4,Ord_5484,322.82,-17.58,False,True
...,...,...,...,...,...
8394,Ord_5018,7325.63,1899.23,True,False
8395,Ord_669,20872.16,-4437.91,False,True
8396,Ord_508,19109.61,-379.29,False,True
8397,Ord_3721,614.14,-735.27,False,True


In [10]:
master_df.loc[master_df.Profit > 0]

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin,...,Ship_Mode,Ship_Date,Order_ID_y,Order_Date,Order_Priority,is_profitable,Is_Profit,Is_Profitable,Is_profit_btw_1000_and_2000,has_loss
1,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.69,0.00,26,1148.90,2.50,0.59,...,EXPRESS AIR,27-07-2010,36262,27-07-2010,NOT SPECIFIED,True,True,True,True,False
3,Ord_2978,Prod_16,SHP_4112,Cust_1088,305.05,0.04,27,23.12,3.37,0.57,...,REGULAR AIR,26-02-2011,37863,24-02-2011,HIGH,True,True,True,False,False
5,Ord_3730,Prod_16,SHP_5175,Cust_1314,459.08,0.04,34,61.57,3.14,0.60,...,EXPRESS AIR,09-12-2009,36992,07-12-2009,MEDIUM,True,True,True,False,False
7,Ord_4143,Prod_6,SHP_5770,Cust_1417,901.32,0.10,25,338.01,5.08,0.38,...,EXPRESS AIR,11-10-2009,13920,09-10-2009,CRITICAL,True,True,True,False,False
10,Ord_4796,Prod_13,SHP_6687,Cust_1659,150.06,0.02,49,30.97,0.70,0.56,...,REGULAR AIR,19-11-2009,55554,16-11-2009,CRITICAL,True,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8389,Ord_757,Prod_14,SHP_1034,Cust_231,6607.92,0.06,34,1462.72,24.49,0.46,...,REGULAR AIR,16-01-2010,20961,14-01-2010,NOT SPECIFIED,True,True,True,True,False
8390,Ord_1096,Prod_14,SHP_1512,Cust_423,2222.61,0.02,3,196.08,24.49,0.41,...,REGULAR AIR,21-06-2011,38693,14-06-2011,LOW,True,True,True,False,False
8391,Ord_5097,Prod_14,SHP_7119,Cust_1731,21046.74,0.04,29,5217.27,24.49,0.54,...,REGULAR AIR,23-09-2011,21188,21-09-2011,CRITICAL,True,True,True,False,False
8393,Ord_997,Prod_14,SHP_1379,Cust_365,28761.52,0.04,8,285.11,24.49,0.37,...,REGULAR AIR,23-10-2010,48800,22-10-2010,NOT SPECIFIED,True,True,True,False,False


The same can be done in just one line of code using lambda functions. 

In [11]:
# Create a new column using a lambda function
master_df['is_profitable'] = master_df['Profit'].apply(lambda x: x > 0)
master_df.head()

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin,...,Ship_Mode,Ship_Date,Order_ID_y,Order_Date,Order_Priority,is_profitable,Is_Profit,Is_Profitable,Is_profit_btw_1000_and_2000,has_loss
0,Ord_5446,Prod_16,SHP_7609,Cust_1818,136.81,0.01,23,-30.51,3.60,0.56,...,REGULAR AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED,False,False,False,False,True
1,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.69,0.00,26,1148.90,2.50,0.59,...,EXPRESS AIR,27-07-2010,36262,27-07-2010,NOT SPECIFIED,True,True,True,True,False
2,Ord_5446,Prod_6,SHP_7608,Cust_1818,164.02,0.03,23,-47.64,6.15,0.37,...,EXPRESS AIR,28-07-2010,36262,27-07-2010,NOT SPECIFIED,False,False,False,False,True
3,Ord_2978,Prod_16,SHP_4112,Cust_1088,305.05,0.04,27,23.12,3.37,0.57,...,REGULAR AIR,26-02-2011,37863,24-02-2011,HIGH,True,True,True,False,False
4,Ord_5484,Prod_16,SHP_7663,Cust_1820,322.82,0.05,35,-17.58,3.98,0.56,...,REGULAR AIR,03-03-2012,53026,26-02-2012,LOW,False,False,False,False,True


Now you can use the new column to compare the percentage of profitable orders across groups.

In [12]:
# Comparing percentage of profitable orders across customer segments
by_segment = master_df.groupby('Customer_Segment')
by_segment.is_profitable.mean()

Customer_Segment
CONSUMER          0.500910
CORPORATE         0.481469
HOME OFFICE       0.498524
SMALL BUSINESS    0.496346
Name: is_profitable, dtype: float64

In [13]:
df_grp_by_region = master_df.groupby('Region')
df_grp_by_region.is_profitable.mean()

Region
ATLANTIC                 0.503704
NORTHWEST TERRITORIES    0.492386
NUNAVUT                  0.481013
ONTARIO                  0.501643
PRARIE                   0.499414
QUEBEC                   0.460948
WEST                     0.486690
YUKON                    0.483395
Name: is_profitable, dtype: float64

In [14]:
# Comparing percentage of profitable orders across product categories
by_category = master_df.groupby('Product_Category')
by_category.is_profitable.mean()

Product_Category
FURNITURE          0.465197
OFFICE SUPPLIES    0.466161
TECHNOLOGY         0.573366
Name: is_profitable, dtype: float64

In FURNITURE, 46% orders are profitable, compared to 57% in TECHNOLOGY. 

In [15]:
# You can also use apply and lambda to alter existing columns
# E.g. you want to see Profit as one decimal place
# apply the round() function 
master_df['Profit'] = master_df['Profit'].apply(lambda x: round(x, 1))
master_df.loc[:, columns].head()

,Ord_id,Sales,Profit,is_profitable,has_loss
0,Ord_5446,136.81,-30.5,False,True
1,Ord_5446,4701.69,1148.9,True,False
2,Ord_5446,164.02,-47.6,False,True
3,Ord_2978,305.05,23.1,True,False
4,Ord_5484,322.82,-17.6,False,True


In [16]:
master_df['Sales'] = master_df['Sales'].apply(lambda data: round(data, 1))
master_df.loc[:, columns].head()

,Ord_id,Sales,Profit,is_profitable,has_loss
0,Ord_5446,136.8,-30.5,False,True
1,Ord_5446,4701.7,1148.9,True,False
2,Ord_5446,164.0,-47.6,False,True
3,Ord_2978,305.1,23.1,True,False
4,Ord_5484,322.8,-17.6,False,True


You sometimes need to create new columns using existing columns, for instance, say you want a column ```Profit / Order_Quantity```. 

In [17]:
# Creating a column Profit / Order_Quantity
master_df['profit_per_qty'] = master_df['Profit'] / master_df['Order_Quantity']
master_df.head()

,Ord_id,Prod_id,Ship_id,Cust_id,Sales,Discount,Order_Quantity,Profit,Shipping_Cost,Product_Base_Margin,...,Ship_Date,Order_ID_y,Order_Date,Order_Priority,is_profitable,Is_Profit,Is_Profitable,Is_profit_btw_1000_and_2000,has_loss,profit_per_qty
0,Ord_5446,Prod_16,SHP_7609,Cust_1818,136.8,0.01,23,-30.5,3.60,0.56,...,28-07-2010,36262,27-07-2010,NOT SPECIFIED,False,False,False,False,True,-1.326087
1,Ord_5446,Prod_4,SHP_7610,Cust_1818,4701.7,0.00,26,1148.9,2.50,0.59,...,27-07-2010,36262,27-07-2010,NOT SPECIFIED,True,True,True,True,False,44.188462
2,Ord_5446,Prod_6,SHP_7608,Cust_1818,164.0,0.03,23,-47.6,6.15,0.37,...,28-07-2010,36262,27-07-2010,NOT SPECIFIED,False,False,False,False,True,-2.069565
3,Ord_2978,Prod_16,SHP_4112,Cust_1088,305.1,0.04,27,23.1,3.37,0.57,...,26-02-2011,37863,24-02-2011,HIGH,True,True,True,False,False,0.855556
4,Ord_5484,Prod_16,SHP_7663,Cust_1820,322.8,0.05,35,-17.6,3.98,0.56,...,03-03-2012,53026,26-02-2012,LOW,False,False,False,False,True,-0.502857


### Pivot Tables

You may want to use pandas pivot tables as an alternative to ```groupby()```. They provide Excel-like functionalities to create aggregate tables. 

In [18]:
# Read documentation
help(pd.DataFrame.pivot_table)

Help on function pivot_table in module pandas.core.frame:

pivot_table(self, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True) -> 'DataFrame'
    Create a spreadsheet-style pivot table as a DataFrame.
    
    The levels in the pivot table will be stored in MultiIndex objects
    (hierarchical indexes) on the index and columns of the result DataFrame.
    
    Parameters
    ----------
    values : column to aggregate, optional
    index : column, Grouper, array, or list of the previous
        If an array is passed, it must be the same length as the data. The
        list can contain any of the other types (except list).
        Keys to group by on the pivot table index.  If an array is passed,
        it is being used as the same manner as column values.
    columns : column, Grouper, array, or list of the previous
        If an array is passed, it must be the same length as the data. The

The general syntax is ```pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', ...)```.
* ```data``` is a dataframe
* ```values``` contains the column to aggregate
* ```index``` is the row in the pivot table
* ```columns``` contains the columns you want in the pivot table
* ```aggfunc``` is the aggregate function

Let's see some examples.

In [22]:
# E.g. Compare average Sales across customer segments- Using groupBy function
master_df.groupby(['Customer_Segment'])['Sales'].mean()

Customer_Segment
CONSUMER          1857.859127
CORPORATE         1787.680819
HOME OFFICE       1754.312844
SMALL BUSINESS    1698.123569
Name: Sales, dtype: float64

In [45]:
# E.g. Compare average Sales across customer segments - using pivot table
master_df.pivot_table(values = 'Sales', index = 'Customer_Segment', aggfunc = np.sum).sort_values(by='Sales')

,Sales
Customer_Segment,
SMALL BUSINESS,2788318.9
CONSUMER,3063609.7
HOME OFFICE,3564763.7
CORPORATE,5498906.2


In [67]:
master_df.pivot_table(index=['Product_Category', 'Product_Sub_Category'], columns= ['Region'], values=['Profit'], 
                      aggfunc= {'Profit': np.sum})

Profit  \
Region                                          ATLANTIC   
Product_Category Product_Sub_Category                      
FURNITURE        BOOKCASES                        7887.0   
                 CHAIRS & CHAIRMATS              13177.8   
                 OFFICE FURNISHINGS               6576.1   
                 TABLES                         -12295.0   
OFFICE SUPPLIES  APPLIANCES                      24516.7   
                 BINDERS AND BINDER ACCESSORIES  36529.4   
                 ENVELOPES                        4256.2   
                 LABELS                           2138.5   
                 PAPER                           11074.5   
                 PENS & ART SUPPLIES               747.2   
                 RUBBER BANDS                       78.7   
                 SCISSORS, RULERS AND TRIMMERS    -792.2   
                 STORAGE & ORGANIZATION         -11578.7   
TECHNOLOGY       COMPUTER PERIPHERALS            14475.5   
                 COPIERS AND FAX                 12885.3   
                 OFFICE MACHINES                 88127.2   
                 TELEPHONES AND COMMUNICATION    41157.8   

                                                                               \
Region                                          NORTHWEST TERRITORIES NUNAVUT   
Product_Category Product_Sub_Category                                           
FURNITURE        BOOKCASES                                   -14323.1   102.4   
                 CHAIRS & CHAIRMATS                           12099.4  3372.4   
                 OFFICE FURNISHINGS                            5843.4   484.9   
                 TABLES                                        3831.8  -481.0   
OFFICE SUPPLIES  APPLIANCES                                    4764.1   451.4   
                 BINDERS AND BINDER ACCESSORIES               30060.1  -417.5   
                 ENVELOPES                                     3566.7   447.0   
                 LABELS                                         557.0   254.2   
                 PAPER                                         1516.4  -338.0   
                 PENS & ART SUPPLIES                            224.7    33.1   
                 RUBBER BANDS                                   128.5   -20.1   
                 SCISSORS, RULERS AND TRIMMERS                 -172.3 -1759.6   
                 STORAGE & ORGANIZATION                       -3651.1 -1774.3   
TECHNOLOGY       COMPUTER PERIPHERALS                          -402.3  1098.1   
                 COPIERS AND FAX                              10976.3  -547.6   
                 OFFICE MACHINES                              27455.7     NaN   
                 TELEPHONES AND COMMUNICATION                 18177.7  1935.9   

                                                                             \
Region                                           ONTARIO    PRARIE   QUEBEC   
Product_Category Product_Sub_Category                                         
FURNITURE        BOOKCASES                      -15036.9    3665.8 -12369.4   
                 CHAIRS & CHAIRMATS              56447.3    7003.9  29832.4   
                 OFFICE FURNISHINGS              20898.5   25464.0  10076.7   
                 TABLES                         -42421.9   -5582.0 -28300.7   
OFFICE SUPPLIES  APPLIANCES                      29854.0   11653.1   7441.6   
                 BINDERS AND BINDER ACCESSORIES  94699.0   42303.0  30321.5   
                 ENVELOPES                        8060.3    8118.1   4963.1   
                 LABELS                           3389.7    2673.3   1143.6   
                 PAPER                           13420.0    5256.7   3639.4   
                 PENS & ART SUPPLIES              2128.7    1336.4   1233.7   
                 RUBBER BANDS                      152.9     -54.4   -227.0   
                 SCISSORS, RULERS AND TRIMMERS   -2023.1   -1968.1   -692.3   
                 STORAGE & ORGANIZATION    

In [68]:
master_df.columns

Index(['Ord_id', 'Prod_id', 'Ship_id', 'Cust_id', 'Sales', 'Discount',
       'Order_Quantity', 'Profit', 'Shipping_Cost', 'Product_Base_Margin',
       'Customer_Name', 'Province', 'Region', 'Customer_Segment',
       'Product_Category', 'Product_Sub_Category', 'Order_ID_x', 'Ship_Mode',
       'Ship_Date', 'Order_ID_y', 'Order_Date', 'Order_Priority',
       'is_profitable', 'Is_Profit', 'Is_Profitable',
       'Is_profit_btw_1000_and_2000', 'has_loss', 'profit_per_qty'],
      dtype='object')

In [69]:
master_df.pivot_table(index=['Product_Category', 'Product_Sub_Category'], values=['Sales', 'Profit'], aggfunc='mean', sort=True)

Profit         Sales
Product_Category Product_Sub_Category                                     
FURNITURE        BOOKCASES                       -177.686772   4352.656085
                 CHAIRS & CHAIRMATS               387.693264   4564.345078
                 OFFICE FURNISHINGS               127.446447    885.904569
                 TABLES                          -274.409141   5252.098061
OFFICE SUPPLIES  APPLIANCES                       223.869124   1698.138249
                 BINDERS AND BINDER ACCESSORIES   335.972131   1117.986448
                 ENVELOPES                        195.861789    707.666667
                 LABELS                            47.490972    135.356944
                 PAPER                             36.948735    364.450367
                 PENS & ART SUPPLIES               11.952923    263.994313
                 RUBBER BANDS                      -0.570950     83.834637
                 SCISSORS, RULERS AND TRIMMERS    -54.157639    562.470833
                 STORAGE & ORGANIZATION            12.203297   1960.039560
TECHNOLOGY       COMPUTER PERIPHERALS             124.390897   1049.967414
                 COPIERS AND FAX                 1923.693103  12992.657471
                 OFFICE MACHINES                  913.094955   6435.302671
                 TELEPHONES AND COMMUNICATION     358.950510   2139.652661

In [70]:
master_df.pivot_table(index=['Product_Category', 'Product_Sub_Category']).loc[: ,['Sales', 'Profit']]

Sales       Profit
Product_Category Product_Sub_Category                                     
FURNITURE        BOOKCASES                        4352.656085  -177.686772
                 CHAIRS & CHAIRMATS               4564.345078   387.693264
                 OFFICE FURNISHINGS                885.904569   127.446447
                 TABLES                           5252.098061  -274.409141
OFFICE SUPPLIES  APPLIANCES                       1698.138249   223.869124
                 BINDERS AND BINDER ACCESSORIES   1117.986448   335.972131
                 ENVELOPES                         707.666667   195.861789
                 LABELS                            135.356944    47.490972
                 PAPER                             364.450367    36.948735
                 PENS & ART SUPPLIES               263.994313    11.952923
                 RUBBER BANDS                       83.834637    -0.570950
                 SCISSORS, RULERS AND TRIMMERS     562.470833   -54.157639
                 STORAGE & ORGANIZATION           1960.039560    12.203297
TECHNOLOGY       COMPUTER PERIPHERALS             1049.967414   124.390897
                 COPIERS AND FAX                 12992.657471  1923.693103
                 OFFICE MACHINES                  6435.302671   913.094955
                 TELEPHONES AND COMMUNICATION     2139.652661   358.950510

In [71]:
master_df.groupby(['Product_Category', 'Product_Sub_Category'])[['Sales', 'Profit']].mean()

Sales       Profit
Product_Category Product_Sub_Category                                     
FURNITURE        BOOKCASES                        4352.656085  -177.686772
                 CHAIRS & CHAIRMATS               4564.345078   387.693264
                 OFFICE FURNISHINGS                885.904569   127.446447
                 TABLES                           5252.098061  -274.409141
OFFICE SUPPLIES  APPLIANCES                       1698.138249   223.869124
                 BINDERS AND BINDER ACCESSORIES   1117.986448   335.972131
                 ENVELOPES                         707.666667   195.861789
                 LABELS                            135.356944    47.490972
                 PAPER                             364.450367    36.948735
                 PENS & ART SUPPLIES               263.994313    11.952923
                 RUBBER BANDS                       83.834637    -0.570950
                 SCISSORS, RULERS AND TRIMMERS     562.470833   -54.157639
                 STORAGE & ORGANIZATION           1960.039560    12.203297
TECHNOLOGY       COMPUTER PERIPHERALS             1049.967414   124.390897
                 COPIERS AND FAX                 12992.657471  1923.693103
                 OFFICE MACHINES                  6435.302671   913.094955
                 TELEPHONES AND COMMUNICATION     2139.652661   358.950510

In [73]:
# E.g. compare total number of profitable orders across regions
# Note that since is_profitable is 1/0, we can directly compute the sum
master_df.pivot_table(values = 'is_profitable', index = 'Region', aggfunc = 'sum').sort_values(by=['is_profitable'])

,is_profitable
Region,
NUNAVUT,38
NORTHWEST TERRITORIES,194
YUKON,262
QUEBEC,360
ATLANTIC,544
PRARIE,852
ONTARIO,916
WEST,969


In [74]:
master_df.columns

Index(['Ord_id', 'Prod_id', 'Ship_id', 'Cust_id', 'Sales', 'Discount',
       'Order_Quantity', 'Profit', 'Shipping_Cost', 'Product_Base_Margin',
       'Customer_Name', 'Province', 'Region', 'Customer_Segment',
       'Product_Category', 'Product_Sub_Category', 'Order_ID_x', 'Ship_Mode',
       'Ship_Date', 'Order_ID_y', 'Order_Date', 'Order_Priority',
       'is_profitable', 'Is_Profit', 'Is_Profitable',
       'Is_profit_btw_1000_and_2000', 'has_loss', 'profit_per_qty'],
      dtype='object')

In [83]:
# index means groupby
master_df.pivot_table(index=['Region', 'Province'], values=['Sales'], aggfunc='mean', sort=True, 
                      margins= True, margins_name='Total')

Sales
Region                Province                          
ATLANTIC              NEW BRUNSWICK          2118.300619
                      NEWFOUNDLAND           1255.170732
                      NOVA SCOTIA            1762.348922
                      PRINCE EDWARD ISLAND   1940.202370
NORTHWEST TERRITORIES NORTHWEST TERRITORIES  2032.608122
NUNAVUT               NUNAVUT                1473.118987
ONTARIO               ONTARIO                1677.552355
PRARIE                MANITOBA               1731.210340
                      SASKACHEWAN            1604.002629
QUEBEC                QUEBEC                 1933.668118
WEST                  ALBERTA                1970.857110
                      BRITISH COLUMBIA       1680.957993
YUKON                 YUKON                  1800.492804
Total                                        1775.877902

In [63]:
# Grouping by both rows and columns
# Compare the total profit across product categories and customer segments
# Since there are two categorical variables, we use both rows (index) and columns
master_df.pivot_table(values = 'Profit', 
                      index = 'Product_Category', 
                      columns = 'Customer_Segment', 
                      aggfunc = 'sum')

Customer_Segment,CONSUMER,CORPORATE,HOME OFFICE,SMALL BUSINESS
Product_Category,,,,
FURNITURE,42728.5,22008.3,23978.6,28717.5
OFFICE SUPPLIES,88532.4,203038.8,121145.6,105306.8
TECHNOLOGY,156700.1,374701.1,173230.6,181684.1


In [84]:
master_df.pivot_table(values=['Sales', 'Profit'],
                      index=['Product_Category'],
                      columns=['Customer_Segment'],
                      aggfunc='sum')

Profit                                           Sales  \
Customer_Segment  CONSUMER CORPORATE HOME OFFICE SMALL BUSINESS   CONSUMER   
Product_Category                                                             
FURNITURE          42728.5   22008.3     23978.6        28717.5  1128806.7   
OFFICE SUPPLIES    88532.4  203038.8    121145.6       105306.8   691382.1   
TECHNOLOGY        156700.1  374701.1    173230.6       181684.1  1243420.9   

                                                        
Customer_Segment  CORPORATE HOME OFFICE SMALL BUSINESS  
Product_Category                                        
FURNITURE         1862840.1   1285346.2       901596.4  
OFFICE SUPPLIES   1341316.8    960054.2       760009.2  
TECHNOLOGY        2294749.3   1319363.3      1126713.3

You don't necessarily need to specify all four arguments, since ```pivot_table()``` has some smart defaults. For instance, if you just provide ```columns```, it will compute the **mean of all the numeric columns** across each column. For e.g.:

In [67]:
# Computes the mean of all numeric columns across categories
# Notice that the means of Order_IDs are meaningless
master_df.pivot_table(columns = 'Product_Category')

Product_Category,FURNITURE,OFFICE SUPPLIES,TECHNOLOGY
Discount,0.049287,0.050230,0.048746
Order_ID_x,30128.711717,30128.122560,29464.891525
Order_ID_y,30128.711717,30128.122560,29464.891525
Order_Quantity,25.709977,25.656833,25.266344
Product_Base_Margin,0.598555,0.461270,0.556305
Profit,68.116531,112.369544,429.208668
Sales,3003.822158,814.048221,2897.940339
Shipping_Cost,30.883811,7.829829,8.954886
has_loss,0.534803,0.533839,0.426634
is_profitable,0.465197,0.466161,0.573366


In [86]:
master_df.pivot_table(index = 'Product_Category').sort_values(by='Profit')

,Discount,Is_Profit,Is_Profitable,Is_profit_btw_1000_and_2000,Order_ID_x,Order_ID_y,Order_Quantity,Product_Base_Margin,Profit,Sales,Shipping_Cost,has_loss,is_profitable,profit_per_qty
Product_Category,,,,,,,,,,,,,,
FURNITURE,0.049287,0.465197,0.465197,0.055104,30128.711717,30128.711717,25.709977,0.598555,68.116531,3003.822158,30.883811,0.534803,0.465197,-3.607020
OFFICE SUPPLIES,0.050230,0.466161,0.466161,0.019957,30128.122560,30128.122560,25.656833,0.461270,112.369544,814.048221,7.829829,0.533839,0.466161,1.736175
TECHNOLOGY,0.048746,0.573366,0.573366,0.090557,29464.891525,29464.891525,25.266344,0.556305,429.208668,2897.940339,8.954886,0.426634,0.573366,-52.274216


In [88]:
master_df.pivot_table(columns=['Customer_Segment'])

Customer_Segment,CONSUMER,CORPORATE,HOME OFFICE,SMALL BUSINESS
Discount,0.049903,0.049841,0.049444,0.049403
Is_Profit,0.500910,0.481469,0.498524,0.496346
Is_Profitable,0.500910,0.481469,0.498524,0.496346
Is_profit_btw_1000_and_2000,0.036992,0.045839,0.050197,0.042631
Order_ID_x,29651.300788,29526.969766,30227.655020,30776.489647
Order_ID_y,29651.300788,29526.969766,30227.655020,30776.489647
Order_Quantity,25.324439,25.525683,25.987697,25.391596
Product_Base_Margin,0.512905,0.512783,0.513608,0.510258
Profit,174.627653,194.976658,156.670669,192.270646
Sales,1857.859127,1787.680819,1754.312844,1698.123569


In [89]:
master_df.pivot_table(index=['Customer_Segment'])

,Discount,Is_Profit,Is_Profitable,Is_profit_btw_1000_and_2000,Order_ID_x,Order_ID_y,Order_Quantity,Product_Base_Margin,Profit,Sales,Shipping_Cost,has_loss,is_profitable,profit_per_qty
Customer_Segment,,,,,,,,,,,,,,
CONSUMER,0.049903,0.500910,0.500910,0.036992,29651.300788,29651.300788,25.324439,0.512905,174.627653,1857.859127,13.024748,0.499090,0.500910,-13.513217
CORPORATE,0.049841,0.481469,0.481469,0.045839,29526.969766,29526.969766,25.525683,0.512783,194.976658,1787.680819,12.698911,0.518531,0.481469,-17.050455
HOME OFFICE,0.049444,0.498524,0.498524,0.050197,30227.655020,30227.655020,25.987697,0.513608,156.670669,1754.312844,12.771757,0.501476,0.498524,-9.194568
SMALL BUSINESS,0.049403,0.496346,0.496346,0.042631,30776.489647,30776.489647,25.391596,0.510258,192.270646,1698.123569,12.995840,0.503654,0.496346,-7.763176
